# **Chapter 3 - Chat with Your Documents  (Backend Code)**




In this chapter we are going to learn how we can chat with our documents.

# Lesson A: Conversational Model Interaction with Environment Setup (doc_bot.py)

**Importing and Environment Setup**: Imports necessary modules and components, including environment variables (load_env), text processing and modeling components (RecursiveCharacterTextSplitter, OpenAIEmbeddings, FAISS, S3FileLoader, load_memory_to_pass), callback functions, and garbage collection (gc).

In [ ]:
from .load_env import *
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import S3FileLoader
from .db_utils import (
    load_memory_to_pass,
)
from langchain.callbacks import get_openai_callback
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
import gc

**Generate a Response using a Conversational Model**: Defines a function (get_response) to generate a response using a conversational model.  Takes parameters such as file name, session ID, user query, model name, and temperature for response randomness.

In [ ]:
def get_response(
    file_name: str,
    session_id: str,
    query: str,
    model: str = "gpt-3.5-turbo-16k",
    temperature: float = 0,
):

**Function Workflow Overview**:The function performs a series of steps, including loading embeddings, downloading data from AWS S3, splitting data to comply with GPT token limits, storing data in a vector database, initializing an OpenAI model, creating a conversational retrieval chain, interacting with the model, printing token usage information, storing this information in the answer, and finally, collecting garbage from memory.

In [ ]:
embeddings = OpenAIEmbeddings()  # load embeddings
loader = S3FileLoader(...)  # download file from S3
data = loader.load()  # load data
text_splitter = RecursiveCharacterTextSplitter(...)  # split data
all_splits = text_splitter.split_documents(data)  # store data in vector DB
vectorstore = FAISS.from_documents(all_splits, embeddings)  # initialize vector store
llm = ChatOpenAI(...)  # initialize OpenAI model
qa_chain = ConversationalRetrievalChain.from_llm(...)  # initialize retrieval chain
with get_openai_callback() as cb:  # use callback to determine tokens used
    answer = qa_chain({...})  # interact with the model
print(f"Total Tokens: {cb.total_tokens}")  # print token usage information
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")
answer["total_tokens_used"] = cb.total_tokens  # store token usage in the answer
gc.collect()  # collect garbage from memory
return answer  # return the generated response


#Lesson B:  Environment Variable Configuration for System Integration (load_env.py)


**OS Check**  - Checks if the operating system is Windows (nt stands for Windows). This condition is used to handle environment variable loading differently for Windows.

In [ ]:
# Import the os module for system-related operations
import os

# Check if the operating system is Windows
if os.name == "nt":  # Windows

    # If it's Windows, import the `load_dotenv` function from the `dotenv` library
    from dotenv import load_dotenv

    # Load environment variables from a `.secrets.env` file (used for local development)
    load_dotenv(".secrets.env")

**Retrieving and Assigning Environment Variables**:
S3_KEY = os.environ.get("S3_KEY")
S3_SECRET = os.environ.get("S3_SECRET")
S3_BUCKET = os.environ.get("S3_BUCKET")
S3_REGION = os.environ.get("S3_REGION")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
MONGO_URL = os.environ.get("MONGO_URL")
S3_PATH = os.environ.get("S3_PATH")


In [ ]:
S3_KEY = os.environ.get("S3_KEY")
S3_SECRET = os.environ.get("S3_SECRET")
S3_BUCKET = os.environ.get("S3_BUCKET")
S3_REGION = os.environ.get("S3_REGION")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
MONGO_URL = os.environ.get("MONGO_URL")
S3_PATH = os.environ.get("S3_PATH")

#Lesson C: MongoDB Connection and Initialization with Error Handling (load_db.py)



**Importing Necessary Libraries:** Imports the required libraries for MongoDB interaction (pymongo), loads the MongoDB URL from a local module, and includes modules for error handling (traceback, os, sys).

In [ ]:
import pymongo
from .load_env import MONGO_URL
import traceback
import os, sys

**Connecting to MongoDB and Initializing Collection**: Tries to connect to MongoDB using the provided URL, accesses the "chat_with_doc" database, initializes the "chat-history" collection, and creates an index on the "session_id" field for uniqueness. If an exception occurs, it prints detailed error information, including the exception type, filename, and line number.

In [ ]:
try:
    client = pymongo.MongoClient(MONGO_URL, uuidRepresentation="standard")
    db = client["chat_with_doc"]
    conversationcol = db["chat-history"]
    conversationcol.create_index([("session_id")], unique=True)
except:
    print(traceback.format_exc())
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    print(exc_type, fname, exc_tb.tb_lineno)

# Lesson D: Setup Database (MongoDB) to store and retrieve these conversations (db_utils.py)

**Import Statement**: Imports the conversationcol object from the load_db module, suggesting that load_db likely holds a MongoDB collection object. Additionally, it imports the uuid module for generating unique session IDs and the List type from the typing module, indicating that the load_memory_to_pass function expects a list as one of its parameters.




In [ ]:
from .load_db import conversationcol
import uuid
from typing import List


**Document Retrieval and History Initialization**:
Retrieves a MongoDB document based on the provided session ID and initializes an empty list (history) to store conversation history. The conversation data is then processed, printed, and returned as a list.


In [ ]:
def load_memory_to_pass(session_id: str):
    data = conversationcol.find_one({"session_id": session_id})
    history = []
    if data:
        data = data["conversation"]
        for x in range(0, len(data), 2):
            history.extend([(data[x], data[x + 1])])
    print(history)
    return history

**Generate a New Unique Session ID**


In [ ]:
def get_session() -> str:
    return str(uuid.uuid4())

**Document Handling and Update Logic:**
The code retrieves a MongoDB document based on the provided session ID, checks for its existence, and updates the conversation field with new values if it exists using $set and update_one. If the document doesn't exist, a new document is inserted with the provided session ID and conversation history.

In [ ]:
def add_session_history(session_id: str, new_values: List):
    document = conversationcol.find_one({"session_id": session_id})
    if document:
        conversation = document["conversation"]
        conversation.extend(new_values)
        conversationcol.update_one(
            {"session_id": session_id},
            {"$set": {"conversation": conversation}},
        )
    else:
        conversationcol.insert_one(
            {
                "session_id": session_id,
                "conversation": new_values,
            }
        )

# Lesson 1: Import Modules and Libraries

**Model Imports:** *from model import **  imports components from the model module. This likely includes data models or structures used in the application.

**System Operations:** *import sys* and import traceback are for system-related operations and error handling, respectively.


In [ ]:
from model import *  # Import the model components
import sys  # Import the sys module for system-related operations
import traceback  # Import traceback for error handling

**FastAPI Framework Components:** Import necessary components from the FastAPI framework. FastAPI is the main class for creating the web application, while UploadFile, status, and HTTPException are used for handling file uploads, HTTP status codes, and exceptions, respectively.

In [ ]:
from fastapi import (
    FastAPI,
    UploadFile,
    status,
    HTTPException,
)

**FastAPI Response Components:**This line imports JSONResponse from FastAPI. This is used for constructing and returning JSON responses from the web application.

**CORS Middleware:** CORS (Cross-Origin Resource Sharing) middleware is imported from FastAPI. It is used to handle cross-origin requests by adding appropriate headers to the responses, allowing the web application to be accessed from different domains.

In [ ]:

# Import FastAPI components for building the web application
from fastapi.responses import JSONResponse  # Import JSONResponse for returning JSON responses
from fastapi.middleware.cors import CORSMiddleware  # Import CORS middleware to handle Cross-Origin Resource Sharing

**Custom Module Imports:** These lines import custom modules (doc_bot and db_utils) that contain functions (get_response, add_session_history, load_memory_to_pass) used for bot functionality and managing database sessions.

In [ ]:
# Import custom modules for bot functionality
from bot.doc_bot import get_response  # Import a function for generating responses
from bot.db_utils import (
    add_session_history,
    load_memory_to_pass,
)

**Utility Functions for Database:**The script imports awswrangler for working with AWS services, and the os module for system-related operations.

In [ ]:
# Import utility functions for managing the database
import awswrangler as wr  # Import AWS Wrangler for working with AWS services
import os  # Import the os module for system-related operations

**AWS SDK, Environment Variables and Database Utilities**:The code imports boto3, a Python SDK for AWS services, and load_env module for loading environment variables and configurations.

The last two lines import functions (get_session) for managing bot sessions and modules for loading the database (load_db).

In [ ]:
import boto3  # Import the boto3 library for interacting with AWS services
from bot.load_env import *  # Import environment variables and configurations
from bot.db_utils import get_session  # Import a function for managing bot sessions
from bot.load_db import *  # Import modules for loading the database

# Lesson 2: Setting Up CORS Middleware for Cross-Origin Resource Sharing

**Initialize a FastAPI application** by creating an instance of the FastAPI class. This app instance will serve as the main entry point for defining routes and handling incoming requests.

In [ ]:
# Create a FastAPI application
app = FastAPI()

**Adding CORS Middleware:** Here, we configure CORS middleware to handle Cross-Origin Resource Sharing. The CORSMiddleware is added to the FastAPI application, allowing requests from any origin (allow_origins=["*"]), supporting the inclusion of credentials, allowing all HTTP methods, and accepting any HTTP headers. This ensures seamless communication with the application from different sources.

In [ ]:
# Add CORS middleware to handle Cross-Origin Resource Sharing
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# **Lesson 3: Managing AWS S3 Sessions and Handling Chat Sessions**

**Creating an AWS S3 Session:** In this section, we create an AWS S3 session using the boto3 library. The aws_s3 variable represents the session configured with AWS access credentials (ID and secret) and a specific region (us-east-2). This session allows the application to interact with the AWS S3 storage, enabling file uploads and other operations.

In [ ]:
# Create an AWS S3 session with provided access credentials
aws_s3 = boto3.Session(
    aws_access_key_id=S3_KEY,
    aws_secret_access_key=S3_SECRET,
    region_name="us-east-2",
)

**Handling Chat Messages (Route "/chat"):**
This section defines a route /chat for handling incoming POST requests. The function create_chat_message takes a parameter chats of type ChatMessageSent, representing a data model for incoming chat messages. The function, marked as async, indicates that it can perform asynchronous operations. The details inside the function process the chat message, generate a response, and return it in JSON format.

In [ ]:

@app.post("/chat")
async def create_chat_message(
    chats: ChatMessageSent,
):
    """
    Create a chat message and obtain a response based on user input and session.

    This route allows users to send chat messages, and it returns responses based on
    the provided input and the associated session. If a session ID is not provided
    in the request, a new session is created. The conversation history is updated, and
    the response, along with the session ID, is returned.

    Args:
        chats (ChatMessageSent): A Pydantic model representing the chat message, including
        session ID, user input, and data source.

    Returns:
        JSONResponse: A JSON response containing the response message and the session ID.

    Raises:
        HTTPException: If an unexpected error occurs during the chat message processing,
        it returns a 204 NO CONTENT HTTP status with an "error" detail.
    """
    try:
        if chats.session_id is None:
            session_id = get_session()

            payload = ChatMessageSent(
                session_id=session_id,
                user_input=chats.user_input,
                data_source=chats.data_source,
            )
            payload = payload.dict()

            response = get_response(
                file_name=payload.get("data_source"),
                session_id=payload.get("session_id"),
                query=payload.get("user_input"),
            )

            add_session_history(
                session_id=session_id,
                new_values=[payload.get("user_input"), response["answer"]],
            )

            return JSONResponse(
                content={
                    "response": response,
                    "session_id": str(session_id),
                }
            )

        else:
            payload = ChatMessageSent(
                session_id=str(chats.session_id),
                user_input=chats.user_input,
                data_source=chats.data_source,
            )
            payload = payload.dict()

            response = get_response(
                file_name=payload.get("data_source"),
                session_id=payload.get("session_id"),
                query=payload.get("user_input"),
            )

            add_session_history(
                session_id=str(chats.session_id),
                new_values=[payload.get("user_input"), response["answer"]],
            )

            return JSONResponse(
                content={
                    "response": response,
                    "session_id": str(chats.session_id),
                }
            )
    except Exception:
        print(traceback.format_exc())

        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        raise HTTPException(status_code=status.HTTP_204_NO_CONTENT, detail="error")


# **Lesson 4: Uploading Files to AWS S3 and Error Handling**

**Uploading a File (Route "/uploadFile"):**

This section defines a route /uploadFile for handling file uploads. The function uploadtos3 takes a parameter data_file of type UploadFile, representing an uploaded file. The function is marked as async to handle asynchronous operations. The details inside the function read the file, upload it to the AWS S3 storage, and return information about the uploaded file in JSON format.

In [ ]:
@app.post("/uploadFile")
async def uploadtos3(data_file: UploadFile):
    """
    Uploads a file to Amazon S3 storage.

    This route allows users to upload a file, which is saved temporarily, uploaded to Amazon S3,
    and then removed from the local file system. It returns the filename and S3 file path
    in the response JSON.

    Args:
        data_file (UploadFile): The file to be uploaded.

    Returns:
        JSONResponse: A JSON response containing the filename and S3 file path.

    Raises:
        HTTPException: If the file specified in `data_file` is not found (HTTP status code 404).
    """
    try:
        with open(f"{data_file.filename}", "wb") as out_file:
            content = await data_file.read()  # async read
            out_file.write(content)  # async write
        wr.s3.upload(
            local_file=data_file.filename,
            path=f"s3://{S3_BUCKET}/{S3_PATH}{data_file.filename}",
            boto3_session=aws_s3,
        )
        os.remove(data_file.filename)
        response = {
            "filename": data_file.filename,
            "file_path": f"s3://{S3_BUCKET}/{S3_PATH}{data_file.filename}",
        }

    except FileNotFoundError:
        raise HTTPException(status_code=404, detail="Item not found")

    return JSONResponse(content=response)